In [2]:
import math
import torch
import torch.nn as nn
from builder import *

In [35]:
# 320,320,3

# 320x320x3,1,3


# 1. slice/select [:,:,0] --> 320x320x128,1

# 2. reshape -- [ 320,320,128]

# generate input to test

# test onnx version7

In [48]:
class slice_reshape_operation(nn.Module):
    def __init__(self, op_name, C_in, C_out, img_w, img_h, expansion, stride):
        super(slice_reshape_operation, self).__init__()
        self.img_w = img_w
        self.img_h = img_h
        self.C_in = C_in
        self.op = PRIMITIVES[op_name](C_in, C_out, expansion, stride)
        
    def forward(self, x):
        batch_size = x.shape[0]
        x = x.select(3,0)
        x = x.select(2,0)
        x = torch.reshape(x,(batch_size, self.C_in,  self.img_h, self.img_w))
        #x = x.reshape(batch_size, self.C_in,  self.img_h, self.img_w)
        x = self.op(x)

        return x
        
        

In [49]:
op_name = "ir_k3_re"
C_in = 128
C_out = 128
img_w = 320
img_h = 320
expansion = 6
stride = 1
#generate input to test
input_x = torch.rand(1,img_w*img_h*C_in,1,3)
op = slice_reshape_operation(op_name, C_in, C_out, img_w, img_h, expansion, stride)

y = op(input_x)

In [50]:
torch.onnx.export(op,  # model being run
                  input_x,  
                  "test.onnx",
                  export_params=True,  # store the trained parameter weights inside the model file
                  # the ONNX version to export the model to
                  opset_version=7,
                  verbose=True,
                  input_names=['input'],  # the model's input names
                  output_names=['output'],  # the model's output names
                  dynamic_axes={"input": {0: "batch", 1:"channel",2: "width",3:"height"}}
                 )

graph(%input : Float(*, *, *, *, strides=[39321600, 3, 3, 1], requires_grad=0, device=cpu),
      %45 : Float(768, 128, 1, 1, strides=[128, 1, 1, 1], requires_grad=0, device=cpu),
      %46 : Float(768, strides=[1], requires_grad=0, device=cpu),
      %48 : Float(768, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=0, device=cpu),
      %49 : Float(768, strides=[1], requires_grad=0, device=cpu),
      %51 : Float(128, 768, 1, 1, strides=[768, 1, 1, 1], requires_grad=0, device=cpu),
      %52 : Float(128, strides=[1], requires_grad=0, device=cpu)):
  %19 : Long(4, strides=[1], device=cpu) = onnx::Shape(%input)
  %20 : Long(device=cpu) = onnx::Constant[value={0}]()
  %53 : Tensor = onnx::Cast[to=7](%20)
  %21 : Long(device=cpu) = onnx::Gather[axis=0](%19, %53) # <ipython-input-48-3f51d584589c>:10:0
  %22 : Long(device=cpu) = onnx::Constant[value={0}]()
  %54 : Tensor = onnx::Cast[to=7](%22)
  %23 : Float(*, *, *, strides=[39321600, 3, 3], requires_grad=0, device=cpu) = onnx::Gather[axis=3](

In [39]:
y.shape

torch.Size([1, 128, 320, 320])

In [43]:

inch_outch_imgsize = [(16,32,320),(32,64,160),(32,64,80),(64,128,80),(128,256,30)]
# need to be modified

for item in inch_outch_imgsize:
    input_channel = item[0]
    output_channel = item[1]
    img_size = item[2]
    input = torch.randn(1, input_channel, img_size, img_size)
    for k in PRIMITIVES:
        
        
        op = PRIMITIVES[k](input_channel, output_channel, 6, 1)
        
        # save the pytorch model 
        # input_out_img_opname

        torch.onnx.export(op,  # model being run
                          input,  # model input (or a tuple for multiple inputs)
                          "./onnx_test/{}_{}_{}_{}.onnx".format(input_channel, output_channel, img_size, k),  # where to save the model (can be a file or file-like object)
                          export_params=True,  # store the trained parameter weights inside the model file
                          # the ONNX version to export the model to
                          opset_version=7,
                          verbose=True,
                          input_names=['input'],  # the model's input names
                          output_names=['output'],  # the model's output names
                          dynamic_axes=None,
                         )

graph(%input : Float(1, 16, 320, 320, strides=[1638400, 102400, 320, 1], requires_grad=0, device=cpu)):
  %1 : Float(requires_grad=0, device=cpu) = onnx::Constant[value={0}]()
  %output : Float(1, 16, 320, 320, strides=[1638400, 102400, 320, 1], requires_grad=0, device=cpu) = onnx::Mul(%input, %1)
  return (%output)

graph(%input : Float(1, 16, 320, 320, strides=[1638400, 102400, 320, 1], requires_grad=0, device=cpu),
      %11 : Float(32, 16, 1, 1, strides=[16, 1, 1, 1], requires_grad=0, device=cpu),
      %12 : Float(32, strides=[1], requires_grad=0, device=cpu)):
  %10 : Float(1, 32, 320, 320, strides=[3276800, 102400, 320, 1], requires_grad=1, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[1, 1], pads=[0, 0, 0, 0], strides=[1, 1]](%input, %11, %12)
  %output : Float(1, 32, 320, 320, strides=[3276800, 102400, 320, 1], requires_grad=1, device=cpu) = onnx::Relu(%10) # D:\anaconda\lib\site-packages\torch\nn\functional.py:1296:0
  return (%output)

graph(%input : Floa

graph(%input : Float(1, 16, 320, 320, strides=[1638400, 102400, 320, 1], requires_grad=0, device=cpu),
      %28 : Float(96, 16, 1, 1, strides=[16, 1, 1, 1], requires_grad=0, device=cpu),
      %29 : Float(96, strides=[1], requires_grad=0, device=cpu),
      %31 : Float(96, 1, 5, 5, strides=[25, 25, 5, 1], requires_grad=0, device=cpu),
      %32 : Float(96, strides=[1], requires_grad=0, device=cpu),
      %34 : Float(32, 96, 1, 1, strides=[96, 1, 1, 1], requires_grad=0, device=cpu),
      %35 : Float(32, strides=[1], requires_grad=0, device=cpu)):
  %27 : Float(1, 96, 320, 320, strides=[9830400, 102400, 320, 1], requires_grad=1, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[1, 1], pads=[0, 0, 0, 0], strides=[1, 1]](%input, %28, %29)
  %21 : Float(1, 96, 320, 320, strides=[9830400, 102400, 320, 1], requires_grad=1, device=cpu) = onnx::Relu(%27) # D:\anaconda\lib\site-packages\torch\nn\functional.py:1296:0
  %30 : Float(1, 96, 320, 320, strides=[9830400, 102400, 320, 

graph(%input : Float(1, 16, 320, 320, strides=[1638400, 102400, 320, 1], requires_grad=0, device=cpu),
      %38 : Float(96, 16, 1, 1, strides=[16, 1, 1, 1], requires_grad=0, device=cpu),
      %39 : Float(96, strides=[1], requires_grad=0, device=cpu),
      %41 : Float(96, 1, 7, 7, strides=[49, 49, 7, 1], requires_grad=0, device=cpu),
      %42 : Float(96, strides=[1], requires_grad=0, device=cpu),
      %44 : Float(32, 96, 1, 1, strides=[96, 1, 1, 1], requires_grad=0, device=cpu),
      %45 : Float(32, strides=[1], requires_grad=0, device=cpu)):
  %37 : Float(1, 96, 320, 320, strides=[9830400, 102400, 320, 1], requires_grad=1, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[1, 1], pads=[0, 0, 0, 0], strides=[1, 1]](%input, %38, %39)
  %21 : Float(requires_grad=0, device=cpu) = onnx::Constant[value={3}]()
  %22 : Float(1, 96, 320, 320, strides=[9830400, 102400, 320, 1], device=cpu) = onnx::Add(%37, %21)
  %23 : Float(1, 96, 320, 320, strides=[9830400, 102400, 320, 1]

graph(%input : Float(1, 32, 160, 160, strides=[819200, 25600, 160, 1], requires_grad=0, device=cpu),
      %38 : Float(192, 32, 1, 1, strides=[32, 1, 1, 1], requires_grad=0, device=cpu),
      %39 : Float(192, strides=[1], requires_grad=0, device=cpu),
      %41 : Float(192, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=0, device=cpu),
      %42 : Float(192, strides=[1], requires_grad=0, device=cpu),
      %44 : Float(64, 192, 1, 1, strides=[192, 1, 1, 1], requires_grad=0, device=cpu),
      %45 : Float(64, strides=[1], requires_grad=0, device=cpu)):
  %37 : Float(1, 192, 160, 160, strides=[4915200, 25600, 160, 1], requires_grad=1, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[1, 1], pads=[0, 0, 0, 0], strides=[1, 1]](%input, %38, %39)
  %21 : Float(requires_grad=0, device=cpu) = onnx::Constant[value={3}]()
  %22 : Float(1, 192, 160, 160, strides=[4915200, 25600, 160, 1], device=cpu) = onnx::Add(%37, %21)
  %23 : Float(1, 192, 160, 160, strides=[4915200, 25600, 160, 

graph(%input : Float(1, 32, 160, 160, strides=[819200, 25600, 160, 1], requires_grad=0, device=cpu),
      %28 : Float(192, 32, 1, 1, strides=[32, 1, 1, 1], requires_grad=0, device=cpu),
      %29 : Float(192, strides=[1], requires_grad=0, device=cpu),
      %31 : Float(192, 1, 5, 5, strides=[25, 25, 5, 1], requires_grad=0, device=cpu),
      %32 : Float(192, strides=[1], requires_grad=0, device=cpu),
      %34 : Float(64, 192, 1, 1, strides=[192, 1, 1, 1], requires_grad=0, device=cpu),
      %35 : Float(64, strides=[1], requires_grad=0, device=cpu)):
  %27 : Float(1, 192, 160, 160, strides=[4915200, 25600, 160, 1], requires_grad=1, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[1, 1], pads=[0, 0, 0, 0], strides=[1, 1]](%input, %28, %29)
  %21 : Float(1, 192, 160, 160, strides=[4915200, 25600, 160, 1], requires_grad=1, device=cpu) = onnx::Relu(%27) # D:\anaconda\lib\site-packages\torch\nn\functional.py:1296:0
  %30 : Float(1, 192, 160, 160, strides=[4915200, 25600, 1

graph(%input : Float(1, 32, 80, 80, strides=[204800, 6400, 80, 1], requires_grad=0, device=cpu),
      %28 : Float(192, 32, 1, 1, strides=[32, 1, 1, 1], requires_grad=0, device=cpu),
      %29 : Float(192, strides=[1], requires_grad=0, device=cpu),
      %31 : Float(192, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=0, device=cpu),
      %32 : Float(192, strides=[1], requires_grad=0, device=cpu),
      %34 : Float(64, 192, 1, 1, strides=[192, 1, 1, 1], requires_grad=0, device=cpu),
      %35 : Float(64, strides=[1], requires_grad=0, device=cpu)):
  %27 : Float(1, 192, 80, 80, strides=[1228800, 6400, 80, 1], requires_grad=1, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[1, 1], pads=[0, 0, 0, 0], strides=[1, 1]](%input, %28, %29)
  %21 : Float(1, 192, 80, 80, strides=[1228800, 6400, 80, 1], requires_grad=1, device=cpu) = onnx::Relu(%27) # D:\anaconda\lib\site-packages\torch\nn\functional.py:1296:0
  %30 : Float(1, 192, 80, 80, strides=[1228800, 6400, 80, 1], requires_g

graph(%input : Float(1, 32, 80, 80, strides=[204800, 6400, 80, 1], requires_grad=0, device=cpu),
      %28 : Float(192, 32, 1, 1, strides=[32, 1, 1, 1], requires_grad=0, device=cpu),
      %29 : Float(192, strides=[1], requires_grad=0, device=cpu),
      %31 : Float(192, 1, 7, 7, strides=[49, 49, 7, 1], requires_grad=0, device=cpu),
      %32 : Float(192, strides=[1], requires_grad=0, device=cpu),
      %34 : Float(64, 192, 1, 1, strides=[192, 1, 1, 1], requires_grad=0, device=cpu),
      %35 : Float(64, strides=[1], requires_grad=0, device=cpu)):
  %27 : Float(1, 192, 80, 80, strides=[1228800, 6400, 80, 1], requires_grad=1, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[1, 1], pads=[0, 0, 0, 0], strides=[1, 1]](%input, %28, %29)
  %21 : Float(1, 192, 80, 80, strides=[1228800, 6400, 80, 1], requires_grad=1, device=cpu) = onnx::Relu(%27) # D:\anaconda\lib\site-packages\torch\nn\functional.py:1296:0
  %30 : Float(1, 192, 80, 80, strides=[1228800, 6400, 80, 1], requires

graph(%input : Float(1, 64, 80, 80, strides=[409600, 6400, 80, 1], requires_grad=0, device=cpu),
      %28 : Float(384, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %29 : Float(384, strides=[1], requires_grad=0, device=cpu),
      %31 : Float(384, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=0, device=cpu),
      %32 : Float(384, strides=[1], requires_grad=0, device=cpu),
      %34 : Float(128, 384, 1, 1, strides=[384, 1, 1, 1], requires_grad=0, device=cpu),
      %35 : Float(128, strides=[1], requires_grad=0, device=cpu)):
  %27 : Float(1, 384, 80, 80, strides=[2457600, 6400, 80, 1], requires_grad=1, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[1, 1], pads=[0, 0, 0, 0], strides=[1, 1]](%input, %28, %29)
  %21 : Float(1, 384, 80, 80, strides=[2457600, 6400, 80, 1], requires_grad=1, device=cpu) = onnx::Relu(%27) # D:\anaconda\lib\site-packages\torch\nn\functional.py:1296:0
  %30 : Float(1, 384, 80, 80, strides=[2457600, 6400, 80, 1], requires

graph(%input : Float(1, 64, 80, 80, strides=[409600, 6400, 80, 1], requires_grad=0, device=cpu),
      %28 : Float(384, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %29 : Float(384, strides=[1], requires_grad=0, device=cpu),
      %31 : Float(384, 1, 7, 7, strides=[49, 49, 7, 1], requires_grad=0, device=cpu),
      %32 : Float(384, strides=[1], requires_grad=0, device=cpu),
      %34 : Float(128, 384, 1, 1, strides=[384, 1, 1, 1], requires_grad=0, device=cpu),
      %35 : Float(128, strides=[1], requires_grad=0, device=cpu)):
  %27 : Float(1, 384, 80, 80, strides=[2457600, 6400, 80, 1], requires_grad=1, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[1, 1], pads=[0, 0, 0, 0], strides=[1, 1]](%input, %28, %29)
  %21 : Float(1, 384, 80, 80, strides=[2457600, 6400, 80, 1], requires_grad=1, device=cpu) = onnx::Relu(%27) # D:\anaconda\lib\site-packages\torch\nn\functional.py:1296:0
  %30 : Float(1, 384, 80, 80, strides=[2457600, 6400, 80, 1], requir

graph(%input : Float(1, 128, 30, 30, strides=[115200, 900, 30, 1], requires_grad=0, device=cpu),
      %28 : Float(768, 128, 1, 1, strides=[128, 1, 1, 1], requires_grad=0, device=cpu),
      %29 : Float(768, strides=[1], requires_grad=0, device=cpu),
      %31 : Float(768, 1, 5, 5, strides=[25, 25, 5, 1], requires_grad=0, device=cpu),
      %32 : Float(768, strides=[1], requires_grad=0, device=cpu),
      %34 : Float(256, 768, 1, 1, strides=[768, 1, 1, 1], requires_grad=0, device=cpu),
      %35 : Float(256, strides=[1], requires_grad=0, device=cpu)):
  %27 : Float(1, 768, 30, 30, strides=[691200, 900, 30, 1], requires_grad=1, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[1, 1], pads=[0, 0, 0, 0], strides=[1, 1]](%input, %28, %29)
  %21 : Float(1, 768, 30, 30, strides=[691200, 900, 30, 1], requires_grad=1, device=cpu) = onnx::Relu(%27) # D:\anaconda\lib\site-packages\torch\nn\functional.py:1296:0
  %30 : Float(1, 768, 30, 30, strides=[691200, 900, 30, 1], requires_g

In [3]:
op


IRFBlock(
  (pw): ConvBNRelu(
    (conv): Conv2d(128, 768, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (hswish): Hardswish()
  )
  (dw): ConvBNRelu(
    (conv): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768, bias=False)
    (bn): BatchNorm2d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (hswish): Hardswish()
  )
  (pwl): ConvBNRelu(
    (conv): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)